# PDTB_load_json.py

## 导入数据

In [2]:
import json
import pickle
import gensim
import numpy as np
import tensorflow as tf 

/home/zjdou/anaconda3/envs/TF1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/zjdou/anaconda3/envs/TF1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/zjdou/anaconda3/envs/TF1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/zjdou/anaconda3/envs/TF1/lib/python3.6/site-packages

In [3]:
!pwd 

print(tf.__version__)

/data/zjdou/jupyter/root/DiscourseRelation/CNN_Text_Classification2_base/CNN_Text_Classification2_base
1.8.0


In [4]:
# base_path = '/data/zjdou/jupyter/root/DiscourseRelation/CNN_Text_Classification2_base/CNN_Text_Classification2_base/'
base_path = './'
sense = "Temporal"

# dicList_train = [json.loa ds(line) for line in open("D:/PDTB_r/02_20_Implicit.json")]
dicList_train = [json.loads(line) for line in open(base_path + r"PDTB/one_vs_others_standard/Temporal_vs_others/train_sec_02_20")]
dicList_test = [json.loads(line) for line in open(base_path + r"PDTB/one_vs_others_standard/Temporal_vs_others/test_sec_21_22")]
# dicList_al = [json.loads(line) for line in open(r'D:\experiment\PDTB\PDTB语料\00_24_explicit.json')]
dicList_dev = [json.loads(line) for line in open(base_path + r'PDTB/one_vs_others_standard/Temporal_vs_others/dev_sec_00_01')]

In [5]:
print(dicList_test[0])

{'DocID': 'wsj_2100', 'Arg1': {'RawText': 'and viewers will be given a 900 number to call', 'Lemma': ['and', 'viewer', 'will', 'be', 'give', 'a', '900', 'number', 'to', 'call'], 'Word': ['and', 'viewers', 'will', 'be', 'given', 'a', '900', 'number', 'to', 'call'], 'POS': ['CC', 'NNS', 'MD', 'VB', 'VBN', 'DT', 'CD', 'NN', 'TO', 'VB'], 'NER': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}, 'Arg2': {'RawText': "Callers will be sent educational booklets, and the call's modest cost will be an immediate method of raising money", 'Lemma': ['caller', 'will', 'be', 'send', 'educational', 'booklet', ',', 'and', 'the', 'call', "'s", 'modest', 'cost', 'will', 'be', 'a', 'immediate', 'method', 'of', 'raise', 'money'], 'Word': ['Callers', 'will', 'be', 'sent', 'educational', 'booklets', ',', 'and', 'the', 'call', "'s", 'modest', 'cost', 'will', 'be', 'an', 'immediate', 'method', 'of', 'raising', 'money'], 'POS': ['NNS', 'MD', 'VB', 'VBN', 'JJ', 'NNS', ',', 'CC', 'DT', 'NN', 'POS', 'JJ', 'NN', '

In [6]:
print(len(dicList_test))

1046


In [7]:
# 词汇表
# Google用word2vec预训练了300维的新闻语料的词向量googlenews-vecctors-negative300.bin
model = gensim.models.KeyedVectors.load_word2vec_format(base_path + "model/GoogleNews-vectors-negative300.bin",
                                                        unicode_errors="ignore", binary=True) 

In [8]:
print(list(model.vocab.keys())[:100])      

print(model['china'], len(model['china']))  # key是单词，value是300维的词向量

['</s>', 'in', 'for', 'that', 'is', 'on', '##', 'The', 'with', 'said', 'was', 'the', 'at', 'not', 'as', 'it', 'be', 'from', 'by', 'are', 'I', 'have', 'he', 'will', 'has', '####', 'his', 'an', 'this', 'or', 'their', 'who', 'they', 'but', '$', 'had', 'year', 'were', 'we', 'more', '###', 'up', 'been', 'you', 'its', 'one', 'about', 'would', 'which', 'out', 'can', 'It', 'all', 'also', 'two', 'after', 'first', 'He', 'do', 'time', 'than', 'when', 'We', 'over', 'last', 'new', 'other', 'her', 'people', 'into', 'In', 'our', 'there', 'A', 'she', 'could', 'just', 'years', 'some', 'U.S.', 'three', 'million', 'them', 'what', 'But', 'so', 'no', 'like', 'if', 'only', 'percent', 'get', 'did', 'him', 'game', 'back', 'because', 'now', '#.#', 'before']
[-2.94921875e-01  8.44726562e-02  1.03149414e-02  3.45703125e-01
 -9.37500000e-02  5.54199219e-02  1.31835938e-01 -2.73437500e-01
 -1.18164062e-01  3.12500000e-01 -2.96875000e-01 -3.69140625e-01
  1.00585938e-01 -1.79687500e-01 -1.68945312e-01  1.12792969e-

## 预处理数据

- 注意，train的数据，需要保持 sense 和 Not sense 样本数据量相同

  - 下面是用 count1 < count 来控制 数据量相同的

- 然而，dev 和 test 不需要保证数据量相同

### train数据预处理

In [9]:
arg1_train = []       # pdtb中sense为temporal的arg1的词，以[[1],[2], ...]的形式
arg2_train = []       # pdtb中sense为temporal的arg2的词，以[[1],[2], ...]的形式

arg1_train_pos = []   # 存放arg1词性
arg2_train_pos = []   

labels_train = []     # train的标签(sense)

pos_count = 0         # 是temporal计数
neg_count = 0         # 不是temporal计数
count_all = 0         # 样本总数

word_vocab = []       # 将是temporal的arg1，arg2的词以 ['', '', '', '', ] 的形式
pos_vocab = []        # 将是temporal的arg1，arg2的pos标签(词性)以 ['', '', '', '', ]的形式

word_embed = []       # 输入的所有单词的词向量(300维)
pos_embed = []        # 输入的所有单词词性的词向量(300维)
 

# temporal论元对计数
for r in dicList_train:
    a = 'False'
    for s in r['Sense']:     # 'Sense': ['Temporal.Asynchronous.Precedence', ] (可以有多个)
        if sense in s:       # sense = "Temporal"
            a = "True"
    
    if a == "True":
        pos_count += 1
    else:
        neg_count += 1
    
print("pos_count:", pos_count)
print("neg_count:", neg_count)

count_all = pos_count + neg_count
print("number of all words =", count_all)

# 这里是为了平衡 train数据，让正样本和负样本数量一致
count = min(pos_count, neg_count)    # 是时序 和 不是时序 的数量的小的值
count1 = 0
count2 = 0 

# 设置标签为 "Temporal" 和 "Not Temporal"
for r in dicList_train:
    a = "False"
    for s in r["Sense"]:
        if sense in s:
            a = "True"
    
    if a == "True" and count1 < count:  # 是时序，且count1数不到count
        # 存放每个样本的标签(sense)
        labels_train.append(sense)       # Temporal
        
        # 对每个w（r["Arg1"]["Word"]]）如果w不在词向量表中，返回UNK（unknown），否则返回r["Arg1"]["Word"]]小写
        # [['clad', 'in', 'his', 'trademark', 'black', 'velvet', 'suit', '<UNK/>', 'the', '<UNK/>', 'clarinetist', 'announced', 'that', 'his', 'new', 'album', '<UNK/>', '<UNK/>', 'inner', 'voices', '<UNK/>', '<UNK/>', 'had', 'just', 'been', 'released', '<UNK/>', 'that', 'his', 'family', 'was', 'in', 'the', 'front', 'row', '<UNK/>', '<UNK/>', 'that', 'it', 'was', 'his', 'mother', '<UNK/>', 'birthday', '<UNK/>', 'so', 'he', 'was', 'going', '<UNK/>', 'play', 'her', 'favorite', 'tune', 'from', 'the', 'record'], ['<UNK/>', '<UNK/>', 'introduced', 'his', 'colleagues', '<UNK/>', 'bill', 'douglas', '<UNK/>', '<UNK/>', '<UNK/>', 'an', 'old', 'buddy', 'from', 'yale', '<UNK/>', '<UNK/>', 'jazz', 'bassist', 'eddie', 'gomez'], ['an', 'improvisational', 'section', 'was', 'built', 'around', 'pieces', 'by', '<UNK/>', 'douglas', '<UNK/>', 'beginning', 'with', '<UNK/>', 'golden', 'rain', '<UNK/>', '<UNK/>', '<UNK/>', 'lilting', '<UNK/>', '<UNK/>', 'lead', 'in', '<UNK/>', 'the', 'uptempo', '<UNK/>', 'sky', '<UNK/>', '<UNK/>', 'which', 'gave', '<UNK/>', '<UNK/>', 'the', 'opportunity', '<UNK/>', 'wail', 'in', '<UNK/>', 'high', 'register', '<UNK/>', 'show', 'off', 'his', 'fleet', 'fingers']]
        arg1_train.append(["<UNK/>" if w.lower() not in model.vocab.keys() else w.lower() for w in r['Arg1']["Word"]])
        arg2_train.append(["<UNK/>" if w.lower() not in model.vocab.keys() else w.lower() for w in r['Arg2']["Word"]])
        # 存放 Arg1,Arg2的所有单词
        # word_vocab: ['clad', 'in', 'his', 'trademark', 'black', 'velvet', 'suit', ',', 'the', 'soft-spoken', 'clarinetist', 'announced', 'that', 'his', 'new', 'album', ',', '``', 'inner', 'voices', ',', "''", 'had', 'just', 'been', 'released', ',', 'that', 'his', 'family', 'was', 'in', 'the', 'front', 'row', ',', 'and', 'that', 'it', 'was', 'his', 'mother', "'s", 'birthday', ',', 'so', 'he', 'was', 'going', 'to', 'play', 'her', 'favorite', 'tune', 'from', 'the', 'record', 'he', 'launched', 'into', 'saint-saens', "'s", '``', 'the', 'swan', "''", 'from', '``', 'carnival', 'of', 'the', 'animals', ',', "''", 'a', 'favorite', 'encore', 'piece', 'for', 'cellists', ',', 'with', 'lovely', ',', 'glossy', 'tone', 'and', 'no', 'bite', 'mr.', 'stoltzman', 'introduced', 'his', 'colleagues', ':', 'bill', 'douglas', ',', 'pianist/bassoonist/composer', 'and', 'an', 'old', 'buddy', 'from', 'yale', ',', 'and', 'jazz', 'bassist', 'eddie', 'gomez', 'an', 'improvisational', 'section', 'was', 'built', 'around', 'pieces', 'by', 'mr.', 'douglas', ',', 'beginning', 'with', '``', 'golden', 'rain', ',', "''", 'a', 'lilting', ',', 'laid-back', 'lead', 'in', 'to', 'the', 'uptempo', '``', 'sky', ',', "''", 'which', 'gave', 'mr.', 'stoltzman', 'the', 'opportunity', 'to', 'wail', 'in', 'a', 'high', 'register', 'and', 'show', 'off', 'his', 'fleet', 'fingers', 'an', 'improvisational', 'section', 'was', 'built', 'around', 'pieces', 'by', 'mr.', 'douglas', ',', 'beginning', 'with', '``', 'golden', 'rain', ',', "''", 'a', 'lilting', ',', 'laid-back', 'lead', 'in', 'to', 'the', 'uptempo', '``', 'sky', ',', "''", 'which', 'gave', 'mr.', 'stoltzman', 'the', 'opportunity', 'to', 'wail', 'in', 'a', 'high', 'register', 'and', 'show', 'off', 'his', 'fleet', 'fingers', 'bach', "'s", '``', 'air', "''", 'followed']
        word_vocab.extend([w.lower() for w in r['Arg1']['Word']])
        word_vocab.extend([w.lower() for w in r['Arg2']['Word']])
        
        arg1_train_pos.append(r['Arg1']['POS'])
        arg2_train_pos.append(r['Arg2']['POS'])    
        # 存放 Arg1,Arg2的所有POS(词性)
        # pos_vocab: ['vbn', 'in', 'prp$', 'nn', 'jj', 'nn', 'nn', ',', 'dt', 'jj', 'nn', 'vbd', 'in', 'prp$', 'jj', 'nn', ',', '``', 'jj', 'nns', ',', "''", 'vbd', 'rb', 'vbn', 'vbn', ',', 'in', 'prp$', 'nn', 'vbd', 'in', 'dt', 'jj', 'nn', ',', 'cc', 'in', 'prp', 'vbd', 'prp$', 'nn', 'pos', 'nn', ',', 'rb', 'prp', 'vbd', 'vbg', 'to', 'vb', 'prp$', 'jj', 'nn', 'in', 'dt', 'nn', 'prp', 'vbd', 'in', 'nnp', 'pos', '``', 'dt', 'nn', "''", 'in', '``', 'nnp', 'in', 'dt', 'nns', ',', "''", 'dt', 'jj', 'nn', 'nn', 'in', 'nns', ',', 'in', 'jj', ',', 'jj', 'nn', 'cc', 'dt', 'nn', 'nnp', 'nnp', 'vbd', 'prp$', 'nns', ':', 'nnp', 'nnp', ',', 'nn', 'cc', 'dt', 'jj', 'nn', 'in', 'nnp', ',', 'cc', 'nn', 'nn', 'nnp', 'nnp', 'dt', 'jj', 'nn', 'vbd', 'vbn', 'in', 'nns', 'in', 'nnp', 'nnp', ',', 'vbg', 'in', '``', 'nnp', 'nn', ',', "''", 'dt', 'jj', ',', 'jj', 'nn', 'in', 'to', 'dt', 'jj', '``', 'nnp', ',', "''", 'wdt', 'vbd', 'nnp', 'nnp', 'dt', 'nn', 'to', 'vb', 'in', 'dt', 'jj', 'nn', 'cc', 'vb', 'rp', 'prp$', 'nn', 'nns', 'dt', 'jj', 'nn', 'vbd', 'vbn', 'in', 'nns', 'in', 'nnp', 'nnp', ',', 'vbg', 'in', '``', 'nnp', 'nn', ',', "''", 'dt', 'jj', ',', 'jj', 'nn', 'in', 'to', 'dt', 'jj', '``', 'nnp', ',', "''", 'wdt', 'vbd', 'nnp', 'nnp', 'dt', 'nn', 'to', 'vb', 'in', 'dt', 'jj', 'nn', 'cc', 'vb', 'rp', 'prp$', 'nn', 'nns', 'nnp', 'pos', '``', 'nnp', "''", 'vbd']
        pos_vocab.extend([w.lower() for w in r['Arg1']["POS"]])
        pos_vocab.extend([w.lower() for w in r['Arg2']["POS"]])
        
        count1 += 1
    
    elif a != 'True' and count2 < count:  # 不是时序且count2数不到count
        labels_train.append("Not" + sense)
        arg1_train.append(["<UNK/>" if w.lower() not in model.vocab.keys() else w.lower() for w in r["Arg1"]["Word"]])
        arg2_train.append(["<UNK/>" if w.lower() not in model.vocab.keys() else w.lower() for w in r["Arg2"]["Word"]])
        word_vocab.extend([w.lower() for w in r["Arg1"]["Word"]])
        word_vocab.extend([w.lower() for w in r["Arg2"]["Word"]])
       
        arg1_train_pos.append(r["Arg1"]["POS"])
        arg2_train_pos.append(r["Arg2"]["POS"])
        pos_vocab.extend([w.lower() for w in r["Arg1"]["POS"]])
        pos_vocab.extend([w.lower() for w in r["Arg2"]["POS"]])
        count2 += 1
        
# 正样本 和 负样本 数据量相同！
print("is and not\n")
print(count1, count2)

pos_count: 760
neg_count: 760
number of all words = 1520
is and not

760 760


### dev 和 test 数据预处理

In [10]:
def get_not_balance_data(_dic1):
    global word_vocab, pos_vocab
    arg1 = []
    arg2 = []
    arg1_pos = []
    arg2_pos = []
    labels = []
    for r in _dic1:
        a = 'False'
        for s in r['Sense']:
            if sense in s:
                a = 'True'
        
        if a == "True":
            labels.append(sense)
            arg1.append(["<UNK/>" if w.lower() not in model.vocab.keys() else w.lower() for w in r["Arg1"]["Word"]])
            arg2.append(["<UNK/>" if w.lower() not in model.vocab.keys() else w.lower() for w in r["Arg2"]["Word"]])
            word_vocab.extend([w.lower() for w in r["Arg1"]["Word"]])
            word_vocab.extend([w.lower() for w in r["Arg2"]["Word"]])
            arg1_pos.append(r["Arg1"]["POS"])
            arg2_pos.append(r["Arg2"]["POS"])
            pos_vocab.extend([w.lower() for w in r["Arg1"]["POS"]])
            pos_vocab.extend([w.lower() for w in r["Arg2"]["POS"]])
            
        elif a != "True":
            labels.append("Not" + sense)
            arg1.append(["<UNK/>" if w.lower() not in model.vocab.keys() else w.lower() for w in r["Arg1"]["Word"]])
            arg2.append(["<UNK/>" if w.lower() not in model.vocab.keys() else w.lower() for w in r["Arg2"]["Word"]])
            word_vocab.extend([w.lower() for w in r["Arg1"]["Word"]])
            word_vocab.extend([w.lower() for w in r["Arg2"]["Word"]])
            arg1_pos.append(r["Arg1"]["POS"])
            arg2_pos.append(r["Arg2"]["POS"])
            pos_vocab.extend([w.lower() for w in r["Arg1"]["POS"]])
            pos_vocab.extend([w.lower() for w in r["Arg2"]["POS"]])
            
    return arg1, arg2, arg1_pos, arg2_pos, labels

In [11]:
arg1_dev, arg2_dev, arg1_dev_pos, arg2_dev_pos, labels_dev = get_not_balance_data(dicList_dev)
arg1_test, arg2_test, arg1_test_pos, arg2_test_pos, labels_test = get_not_balance_data(dicList_test)

### word_vocab 和 pos_vocab处理

- 开始位置添加 `<PAD/>` 和 `<UNK/>`

In [12]:
print("word_vacab.len, pos_vocab.len, arg1_test.len, arg1_train.len\n ")
print(len(word_vocab))
print(len(pos_vocab))
print(len(arg1_train))

# 在word_vocab \ pos_vocab 开始位置插入<UNK/> <PAD/>
word_vocab.insert(0, '<PAD/>')
pos_vocab.insert(0, '<PAD/>')
word_vocab.insert(0, '<UNK/>')
pos_vocab.insert(0, '<UNK/>')

# 去重
word_vocab = list(set(word_vocab))
pos_vocab = list(set(pos_vocab))

print(len(word_vocab), len(pos_vocab))
print(word_vocab[:10])
print(pos_vocab)

word_vacab.len, pos_vocab.len, arg1_test.len, arg1_train.len
 
142509
142509
1520
15062 46
['swathed', 'jansz', 'write-downs', '9.51', '1.88', 'significance', 'drop', 'elephant', 'insisted', 'sprung']
['wrb', 'vbz', '<PAD/>', 'rp', 'pdt', 'rbr', 'cd', 'wp$', 'jj', 'nn', 'nns', 'rb', 'vbg', 'vb', 'nnps', 'jjs', 'nnp', 'uh', 'fw', 'ex', 'vbp', 'wp', ':', 'pos', 'vbd', 'rbs', '$', 'vbn', 'jjr', 'prp$', ',', 'in', 'prp', '.', 'ls', '<UNK/>', 'dt', 'cc', '-lrb-', 'to', 'md', '#', "''", '-rrb-', 'wdt', '``']


In [13]:
word_embed = []       # 输入的所有单词的词向量(300维)
pos_embed = []        # 输入的所有单词词性的词向量(300维)

for word in word_vocab:
    try:
        word_embed.append(model[word.lower()])
    except:
        # 从一个均匀分布[-1,1)中随机采样300个数，并tolist()将数组转列表（一维列表）
        word_embed.append(np.random.uniform(-1, 1, 300).tolist())
    
print("len of word_embed: ", len(word_embed))
# word_embed.insert(0, [0] * 300)   # insert 后，会比word_vocab多一个成员

for pos in pos_vocab:
    pos_embed.append(np.random.uniform(-1, 1, 300).tolist())
    
print("len of pos_embed: ", len(pos_embed))

len of word_embed:  15062
len of pos_embed:  46


## 存储预处理数据

- dump()方法

 - `pickle.dump(obj, file, [,protocol])`：序列化对象，将对象obj保存到文件file中去。
 
 - 参数protocol是序列化模式，默认是0（ASCII协议，表示以文本的形式进行序列化），protocol的值还可以是1和2（1和2表示以二进制的形式进行序列化。 其中，1是老式的二进制协议；2是新二进制协议）。
 
 - file表示保存到的类文件对象，file必须有write()接口，file可以是一个以'w'打开的文件或者是一个StringIO对象，也可以是任何可以实现write()接口的对象。



- load()方法

 - `pickle.load(file)`

 - 注释：反序列化对象，将文件中的数据解析为一个python对象。file中有read()接口和readline()接口

In [14]:
url_preprocess = base_path + '/preprocess/'

# train
with open(url_preprocess + 'train_temporal_word.pkl', 'wb') as f:
    pickle.dump([arg1_train, arg2_train, labels_train], f)

# dev
with open(url_preprocess + 'dev_temporal_word.pkl', 'wb') as f:
    pickle.dump([arg1_dev, arg2_dev, labels_dev], f)

# test
with open(url_preprocess + 'test_temporal_word.pkl', 'wb') as f:
    pickle.dump([arg1_test, arg2_test, labels_test], f)

with open(url_preprocess + 'test_temporal_pos.pkl', 'wb') as f:
    pickle.dump([arg1_test_pos, arg2_test_pos, labels_test], f)

# pos_vocab_embed 和 word_vocab_embed
with open(url_preprocess + 'pos_vocab_embd_word_vocab_embd_temporal.pkl', 'wb') as f:
    pickle.dump([pos_vocab, pos_embed, word_vocab, word_embed], f)

In [ ]:
# with open(url_preprocess + 'test_temporal_pos.pkl', 'rb') as f:
#     t = pickle.load(f)    # 取数据
#     print(t[2])

# test_cnn.py

- 构建模型

In [15]:
def _variable_on_cpu(name, shape, initializer):
    with tf.device('/cpu:0'):
        var = tf.get_variable(name, shape, initializer=initializer)
        return var

In [16]:
class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(self, sequence_length_arg1, sequence_length_arg2, 
                       word_embedding_size, word_vocab_size, num_classes,
                       filter_sizes, num_filters, l2_reg_lambda = 0.0):
        '''
        :param sequence_length_arg1: 序列1长度
        :param word_embedding_size:  词向量维度
        :param word_vocab_size:      词汇表大小
        :param sequence_length_arg2: 序列2长度
        :param num_classes:          输出的类别数
        :param filter_sizes:         卷积核尺寸
        :param num_filters:          卷积核数目
        :param l2_reg_lambda:        l2正则化超参数
        '''
        # Placeholders for input, output and dropout
        # None: 表示batch_size是任意值
        self.input_x_arg1_word = tf.placeholder(tf.int32, [None, sequence_length_arg1], name='input_x_arg1_word')
        self.input_x_arg2_word = tf.placeholder(tf.int32, [None, sequence_length_arg2], name='input_x_arg2_word')
        self.input_y = tf.placeholder(tf.int64, [None, num_classes], name='input_y')
        # tf是保留结点比例
        self.dropout_keep_prob = tf.placeholder(tf.float32, name='dropout_keep_prob')
        
        # Keeping track of L2 regularization loss (optional)
        l2_loss = tf.constant(0.0)                 # 使用l2正则化的loss
        embedding_size = word_embedding_size
        vocab_size = word_vocab_size
        
        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope('embedding'):
            self._W_emb = _variable_on_cpu(name='embedding', shape=[vocab_size, embedding_size], 
                                           initializer=tf.random_uniform_initializer(minval=-1.0, maxval=1.0))
            # embedding_lookup return tensor
            # tf.nn.embedding_lookup（params, ids）: 选取一个张量里面索引对应的元素; 选取一个张量里面索引对应的元素
            # [batch_size, seq_len, embedding_size]
            self.embedded_chars_arg1 = tf.nn.embedding_lookup(self._W_emb, self.input_x_arg1_word) # get embedding by id
            self.embedded_chars_arg2 = tf.nn.embedding_lookup(self._W_emb, self.input_x_arg2_word)
            
            # 't' is a tensor of shape [2]：shape(expand_dims(t, 0)) ==> [1, 2]; shape(expand_dims(t, -1)) ==> [2, 1] 
            # [batch_size, seq_len, embedding_size, 1]
            self.embedded_chars_expanded_arg1 = tf.expand_dims(self.embedded_chars_arg1, -1)
            self.embedded_chars_expanded_arg2 = tf.expand_dims(self.embedded_chars_arg2, -1)
            
            # Create a convolution + maxpool layer for each filter size
            pooled_output_arg1 = []
            pooled_output_arg2 = []
            
            # filter的种类
            for i, filter_size in enumerate(filter_sizes):
                with tf.name_scope("conv-maxpool-%s" % filter_size):
                    # Convolution layer
                    # 卷积核大小为(filter_size, embedding_dim)
                    # 1表示输入数据通道数，num_filters即输出数据通道数
                    filter_shape = [filter_size, embedding_size, 1, num_filters]
                    
                    # 截断的产生正态分布的随机数，即随机数与均值的差值若大于两倍的标准差，则重新生成。
                    W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name='W') 
                    b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name = 'b')
                    
                    conv_arg1 = tf.nn.conv2d(
                        # input: [batch, in_height, in_weight, in_channel]
                        # [batch, seq_len, embedding_size, 1] 
                        self.embedded_chars_expanded_arg1, 
                        # filter: [filter_height, filter_weight, in_channel, out_channels]
                        # [filter_size, embedding_size, 1, num_filters]
                        W,                       
                        strides = [1, 1, 1, 1],  # strides: [1, strides, strides, 1], 第一位和最后一位固定为1
                        padding = 'VALID',       # padding：值为“SAME” 和 “VALID”,卷积的形式，"SAME"是考虑边界，不足的时候用0去填充周围，"VALID"则不考虑
                        name = 'conv_arg1'
                    )
                    # [b,s,e,1] [fz, e, 1, num_filters] = [b, s, fz, num_filters] ????
                    
                    conv_arg2 = tf.nn.conv2d(
                        self.embedded_chars_expanded_arg2,
                        W,
                        strides = [1, 1, 1, 1],
                        padding = 'VALID',
                        name = 'conv_arg2'
                    )
                    
                    # Apply nonlinearity
                    # activation function [b,s,(1,2,3),1024]
                    # (1,2,3)表示filter_size分别为1,2,3情况，1024为num_filters
                    h_arg1 = tf.nn.relu(tf.nn.bias_add(conv_arg1, b), name='relu_arg1')
                    h_arg2 = tf.nn.relu(tf.nn.bias_add(conv_arg2, b), name='relu_arg2')
            
                    # Maxpooling over the outputs
                    pooled_arg1 = tf.nn.max_pool(
                        h_arg1,
                        ksize = [1, sequence_length_arg1 - filter_size + 1, 1, 1], # 不想在batch和channels上做池化，所以这两个维度设为了1
                        strides = [1, 1, 1, 1],
                        padding = 'VALID',
                        name = 'pool_arg1'
                    )
                    
                    pooled_arg2 = tf.nn.max_pool(
                        h_arg2,
                        ksize=[1, sequence_length_arg2 - filter_size + 1, 1, 1],
                        strides=[1, 1, 1, 1],
                        padding='VALID',
                        name="pool_arg2")
                    
                    pooled_output_arg1.append(pooled_arg1)
                    pooled_output_arg2.append(pooled_arg2)
                    
            # Combine all the pooled features
            num_filters_total = num_filters * len(filter_sizes)
            # 3个filter，由于每个filter输出数据通道num_filters，
            self.h_pool_arg1 = tf.concat(pooled_output_arg1, 3)   # 连接最大池化后
            self.h_pool_arg2 = tf.concat(pooled_output_arg2, 3)   
            
            # 经过flatten可以得到 num_filters*3维向量
            self.h_pool_flat_arg1 = tf.reshape(self.h_pool_arg1, [-1, num_filters_total])
            self.h_pool_flat_arg2 = tf.reshape(self.h_pool_arg2, [-1, num_filters_total])
            self.h_pool_flat = tf.concat([self.h_pool_flat_arg1, self.h_pool_flat_arg2], axis=1, name='h') 
            # [batch, num_filters_total*2]
            
            # Add dropout
            with tf.name_scope("dropout"):
                self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)  # 防止过拟合
                
            # Final (unnormalized) scores and predictions
            # Full connected
            with tf.name_scope("output"):
                W = tf.get_variable(
                    "W",
                    shape=[2 * num_filters_total, num_classes],
                    initializer=tf.contrib.layers.xavier_initializer()
                )
                b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name='b')
                
                # L2正则化
                l2_loss += tf.nn.l2_loss(W)
                l2_loss += tf.nn.l2_loss(b)
                
                # tf.matmul(x, w) + b
                self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name='scores')
                self.predictions = tf.argmax(self.scores, 1, name='predictions') 

            # CaluateMean cross-entropy loss 
            with tf.name_scope('loss'):
                # 第一步是先对网络最后一层的输出做一个softmax
                # 第二步是softmax的输出向量[Y1，Y2,Y3...]和样本的实际标签做一个交叉熵
                self.losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y, name='losses')
                self.loss = tf.reduce_mean(self.losses) + l2_reg_lambda * l2_loss
                
            # Accuracy
            with tf.name_scope('accuracy'):
                # 总体准确率
                correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
                self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, 'float'), name='accuracy')
                
                all_positive_indexes = tf.where(self.predictions > 0)           # 正例的索引
                all_positive_indexes = tf.reshape(all_positive_indexes, [-1])
                
                positive_pred = tf.nn.embedding_lookup(self.predictions, all_positive_indexes)
                positive_label = tf.nn.embedding_lookup(tf.argmax(self.input_y, 1), all_positive_indexes)
                
                # 比较 预测的正例 和 label
                correct_positive_predictions = tf.equal(positive_pred, positive_label)
                
                # Precision值，精确率
                # TP / (TP + FP)
                self.precision = tf.reduce_mean(tf.cast(correct_positive_predictions, dtype=tf.float32), name='precision')
                
                # Recall值，召回率
                # TP / (TP + FN) = TP / P
                # TP: 正确预测正例
                tp = tf.reduce_sum(tf.cast(correct_positive_predictions, dtype=tf.float32))
                
                # P = TP + FN
                self.label_index = tf.cast(tf.argmax(self.input_y, 1), dtype=tf.int32)
                tpfn = tf.reduce_sum(tf.where(self.label_index > 0, 
                                              tf.ones_like(self.label_index, dtype=tf.float32),   # 索引>0, 标志为1
                                              tf.zeros_like(self.label_index, dtype=tf.float32))) # 否则，标志为0
                
                # 函数返回以浮点数进行计算的 x/y.
                self.recall = tf.truediv(tp, tpfn, name='recall')
                
    # 使用预训练词向量
    def assign_embedding(self, session, pretrained_word_embedding):
        session.run(tf.assign(self._W_emb, pretrained_word_embedding))
                

# train.py

- 工具函数

 - data_helpers.py

## data_helpers.py

In [17]:
import numpy as np 
import re 
import itertools
from collections import Counter
from codecs import open
import gensim
import json
import pickle

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()
    
def load_word_pkl(filename):
    with open(filename, 'rb') as file_pkl:
        x_text_arg1, x_text_arg2, labels = pickle.load(file_pkl)
        
    # one-hot：Not temporal--[1, 0], temporal--[0, 1]
    labels = [[1, 0] if "Not" in label else [0, 1] for label in labels]
    
    return x_text_arg1, x_text_arg2, labels


# 处理输入数据
# 1. 填充长度不满足max_document_length的句子
# 2. return the index of the word in the vocab
def build_input_data(sentences, vocab, max_document_length, padding_word="<PAD/>"):
    """
    return the index of the word in the vocab
    """
    padded_sentences = [x[:max_document_length - 1] + [padding_word] * max(max_document_length - len(x), 1) for x in sentences]
    
    x_arg = np.array([[vocab.index(word) for word in sentence] for sentence in padded_sentences])
    
    return x_arg 

# 生成batch数据
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    data = np.array(data)
    
    data_size = len(data)
    
    num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1
    
    for epoch in range(num_epochs):
        # shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffle_data = data[shuffle_indices]
        else:
            shuffle_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffle_data[start_index : end_index]

## train.py

- 这里需要读取命令行参数，写成 .py 文件了

In [18]:
import tensorflow as tf
import numpy as np
import os 
import time
import datetime
# import data_helpers
# from text_cnn import TextCNN
from tensorflow.contrib import learn
import pickle

In [28]:
result_path = base_path + 'experiment/temporal20201009/result1.txt'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"      # 指定gpu版本

# Parameters
# ==================================================

# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 64, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "1,2,3", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 1024, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.2, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
tf.flags.DEFINE_boolean("use_pretrain",True,"use pretrained embedding")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

tf.flags.DEFINE_integer("max_document_length", 80, "Max document length")

FLAGS = tf.flags.FLAGS
# FLAGS._parse_flags()        # 后来版本不用了
FLAGS.flag_values_dict()    

print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))

print("")

DuplicateFlagError: The flag 'dev_sample_percentage' is defined twice. First from /home/zjdou/anaconda3/envs/TF1/lib/python3.6/site-packages/ipykernel_launcher.py, Second from /home/zjdou/anaconda3/envs/TF1/lib/python3.6/site-packages/ipykernel_launcher.py.  Description from first occurrence: Percentage of the training data to use for validation

In [20]:
# Data Preparation
# ==================================================
# load pos file
# x_text_train_arg1_pos, x_text_train_arg2_pos, y_train = data_helpers.load_pos_pkl('./PDTB/one_vs_others_standard/Cause_vs_others/pdtb_with_repetiton_relation_base_pkl/train_cause_pos.pkl')
# x_text_test_arg1_pos, x_text_test_arg2_pos, y_test = data_helpers.load_pos_pkl('./PDTB/one_vs_others_standard/Cause_vs_others/pdtb_with_repetiton_relation_base_pkl/test_cause_pos.pkl')

# load word file
x_text_train_arg1_word, x_text_train_arg2_word, y_train = load_word_pkl('./preprocess/train_temporal_word.pkl')
x_text_test_arg1_word, x_text_test_arg2_word, y_test = load_word_pkl('./preprocess/test_temporal_word.pkl')

with open('./preprocess/pos_vocab_embd_word_vocab_embd_temporal.pkl', 'rb') as file_pkl:
    _, _, word_vocab, word_embd = pickle.load(file_pkl)

In [21]:
print(x_text_test_arg1_word[:5])
print(y_train[:5])

word_embedding_dim = len(word_embd[0])

print(word_embedding_dim)   # 查看词向量维度

[['<UNK/>', 'viewers', 'will', 'be', 'given', '<UNK/>', '<UNK/>', 'number', '<UNK/>', 'call'], ['an', 'assassin', 'in', 'colombia', 'killed', '<UNK/>', 'federal', 'judge', 'on', '<UNK/>', '<UNK/>', 'street'], ['yesterday', '<UNK/>', 'selling', 'began', 'after', '<UNK/>', 'japanese', 'news', 'agency', 'reported', 'that', 'japanese', 'banks', '<UNK/>', 'which', 'balked', 'at', 'the', 'first', 'bid', '<UNK/>', 'were', 'ready', '<UNK/>', 'reject', '<UNK/>', 'revised', 'version', 'at', 'around', '$', '<UNK/>', '<UNK/>', 'share', '<UNK/>', 'or', '$', '<UNK/>', 'billion'], ['the', 'machinists', 'also', 'asked', 'for', 'an', 'investigation', 'by', 'the', 'securities', '<UNK/>', 'exchange', 'commission', 'into', 'possible', '<UNK/>', 'violations', 'in', 'the', 'original', 'bid', 'for', 'ual', 'by', '<UNK/>', 'davis', '<UNK/>', 'as', 'well', 'as', 'in', 'the', 'response', 'by', 'ual'], ['the', 'previous', 'period', '<UNK/>', 'results', 'included', '<UNK/>', '$', '<UNK/>', 'million', 'pretax', 'c

In [23]:
# 每个句子都不一样长，需要把他们pad成一样的长度，并得到索引
x_train_arg1_word = build_input_data(x_text_train_arg1_word, word_vocab, 80)
x_train_arg2_word = build_input_data(x_text_train_arg2_word, word_vocab, 80)
x_test_arg1_word = build_input_data(x_text_test_arg1_word, word_vocab, 80)
x_test_arg2_word = build_input_data(x_text_test_arg2_word, word_vocab, 80)

print(x_train_arg1_word[:2])   # 句子的每个单词在单词表的索引

[[ 8916   429 14714  6863   937  9042   517  4076  3073  4076  5973  2863
   1889 14714 13624  3005  4076  4076 13973  6347  4076  4076  6769  6889
   9243  5582  4076  1889 14714  5843 14070   429  3073 13734  7732  4076
   4076  1889   597 14070 14714 10769  4076  1879  4076  4386 12352 14070
  10888  4076  6197  5213 14090  1779  5720  3073 11882  8976  8976  8976
   8976  8976  8976  8976  8976  8976  8976  8976  8976  8976  8976  8976
   8976  8976  8976  8976  8976  8976  8976  8976]
 [ 4076  4076   721 14714  6104  4076  6763   688  4076  4076  4076  1151
   1919  7389  5720  9185  4076  4076   876 10822 10437  5024  8976  8976
   8976  8976  8976  8976  8976  8976  8976  8976  8976  8976  8976  8976
   8976  8976  8976  8976  8976  8976  8976  8976  8976  8976  8976  8976
   8976  8976  8976  8976  8976  8976  8976  8976  8976  8976  8976  8976
   8976  8976  8976  8976  8976  8976  8976  8976  8976  8976  8976  8976
   8976  8976  8976  8976  8976  8976  8976  8976]]


In [24]:
# pretrained_pos_embedding = np.array(pos_embd)
pretrained_word_embedding = np.array(word_embd)
print(pretrained_word_embedding.shape)

print('data load successfully!')

(15062, 300)
data load successfully!


In [25]:
# Split train/test set
# TODO: This is very crude, should use cross-validation
# dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
# x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
# y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]
# print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Test split: {:d}/{:d}".format(len(y_train), len(y_test)))

np.random.seed(10)
# permutation不直接在原来的数组上进行操作，而是返回一个新的打乱顺序的数组，并不打乱原来的数组
shuffle_indices = np.random.permutation(np.arange(len(y_train)))
# np.nrange作用是一个序列，可被当做向量使用,支持步长为小数

Train/Test split: 1520/1046


In [31]:
# Training
# ==================================================

# tf.app.flags.DEFINE_string('f', '', 'kernel')     # 只定义一次，如果下面代码报错，就运行这个

with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement,
      gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.8))
    
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length_arg1 = x_train_arg1_word.shape[1],
            sequence_length_arg2 = x_train_arg2_word.shape[1],
            word_vocab_size = pretrained_word_embedding.shape[0],
            word_embedding_size = pretrained_word_embedding.shape[1],
            num_classes = np.array(y_train).shape[1],
            filter_sizes = list(map(int, FLAGS.filter_sizes.split(','))),
            num_filters = FLAGS.num_filters,
            l2_reg_lambda = FLAGS.l2_reg_lambda
        )
        print("sequence_length_arg1 :{}\nsequence_length_arg2:{}\nnum_classes:{}\nword_vocab_size:{}\nword_embedding_size:{}\nfilter_sizes:{}\n".
              format(x_train_arg1_word.shape[1], x_train_arg2_word.shape[1], np.array(y_train).shape[1], 
                     len(word_vocab), word_embedding_dim, 
                     list(map(int, FLAGS.filter_sizes.split(",")))))
        if FLAGS.use_pretrain:
            cnn.assign_embedding(sess, pretrained_word_embedding)
        
        # Define Training procedure
        global_step = tf.Variable(0, name='global_step', trainable=False)
        
        # Adam optimizer
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step, name='train_op')
        
        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        # 绘图
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram('{}/grad/hist'.format(v.name), g)
                sparsity_summary = tf.summary.scalar('{}/grad/sparsity'.format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)
        
        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        # 当前路径的绝对路径 + './runs/explict_timestamp'
        out_dir = os.path.abspath(os.path.join(os.path.curdir, 'runs', 'explicit_' + timestamp))
        print('Writing to {}\n'.format(out_dir))
        
        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar('loss', cnn.loss)
        acc_summary = tf.summary.scalar('accuracy', cnn.accuracy)
        
        # Train summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, 'summaries', 'train')
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)
        
        # Dev summary
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, 'summaries', 'dev')
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)
        
        # Checkpoint directory, Tensorflow assumes this directory already exists so we need to create it 
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "explicit-model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)
        
        # Write vocabulary
        # vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())
        
        def train_step(x_batch_arg1_word, x_batch_arg2_word, y_batch):
            '''
            A single training step
            '''
            feed_dict = {
                cnn.input_x_arg1_word: x_batch_arg1_word,
                cnn.input_x_arg2_word: x_batch_arg2_word,
                cnn.input_y: y_batch,
                cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy, precision, recall = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy, cnn.precision, cnn.recall],
                feed_dict
            )
            time_str = datetime.datetime.now().isoformat()
            print("temporal {}: step {}, loss {:g}, acc {:g}, precision {:g}, recall {:g}"
                  .format(time_str, step, loss, accuracy, precision, recall))
            
            train_summary_writer.add_summary(summaries, step)
            
        def dev_step(x_batch_arg1_word, x_batch_arg2_word, y_batch, writer=None):
            '''
            Evaluate model on a dev set
            '''
            feed_dict = {
              cnn.input_x_arg1_word: x_batch_arg1_word,
              cnn.input_x_arg2_word: x_batch_arg2_word,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy, precision, recall = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy, cnn.precision, cnn.recall],
                feed_dict)
            
            time_str = datetime.datetime.now().isoformat()
            
            f1 = 2 * precision * recall / (precision + recall)
            print("temporal {}: step {}, loss {:g}, acc {:g}, precision {:g}, recall {:g}, F1 {:g}"
                  .format(time_str, step, loss, accuracy, precision, recall, f1))
            with open(result_path, "a+") as w_f:
                w_f.write("temporal {}: step {}, loss {:g}, acc {:g}, precision {:g}, recall {:g}, F1 {:g}"
                          .format(time_str, step, loss, accuracy, precision, recall, f1)+"\n")
            if writer:
                writer.add_summary(summaries, step)
            return f1
            
        # Generate batches
        batches = batch_iter(list(zip(x_train_arg1_word, x_train_arg2_word, y_train)), 
                             FLAGS.batch_size, FLAGS.num_epochs)
        
        max_f1 = 0.0
        for batch in batches:
            
            x_batch_arg1_word, x_batch_arg2_word, y_batch = zip(*batch)
            
            train_step(x_batch_arg1_word,x_batch_arg2_word, y_batch)
            
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                f1 = dev_step(x_test_arg1_word, x_test_arg2_word, y_test, writer=dev_summary_writer)
                if f1 > max_f1:
                    max_f1 = f1
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))
                    
                    with open(result_path, "a+") as w_f:
                        w_f.write("max_f1=" + str(max_f1))
                        
                print("")
        

sequence_length_arg1 :80
sequence_length_arg2:80
num_classes:2
word_vocab_size:15062
word_embedding_size:300
filter_sizes:[1, 2, 3]

INFO:tensorflow:Summary name embedding_1:0/grad/hist is illegal; using embedding_1_0/grad/hist instead.
INFO:tensorflow:Summary name embedding_1:0/grad/sparsity is illegal; using embedding_1_0/grad/sparsity instead.
INFO:tensorflow:Summary name embedding/conv-maxpool-1/W:0/grad/hist is illegal; using embedding/conv-maxpool-1/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/conv-maxpool-1/W:0/grad/sparsity is illegal; using embedding/conv-maxpool-1/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name embedding/conv-maxpool-1/b:0/grad/hist is illegal; using embedding/conv-maxpool-1/b_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/conv-maxpool-1/b:0/grad/sparsity is illegal; using embedding/conv-maxpool-1/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name embedding/conv-maxpool-2/W:0/grad/hist is illegal; using embedding/con

/home/zjdou/anaconda3/envs/TF1/lib/python3.6/site-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


temporal 2020-10-10T22:24:43.055404: step 1, loss 4.18811, acc 0.515625, precision 0.5, recall 0.387097
temporal 2020-10-10T22:24:43.324424: step 2, loss 6.19421, acc 0.390625, precision 0.466667, recall 0.583333
temporal 2020-10-10T22:24:43.564212: step 3, loss 4.91019, acc 0.5, precision 0.510204, recall 0.757576
temporal 2020-10-10T22:24:43.804747: step 4, loss 5.29958, acc 0.4375, precision 0.4, recall 0.117647
temporal 2020-10-10T22:24:44.041643: step 5, loss 4.04662, acc 0.5, precision 0.411765, recall 0.241379
temporal 2020-10-10T22:24:44.298926: step 6, loss 3.47414, acc 0.609375, precision 0.566038, recall 0.9375
temporal 2020-10-10T22:24:44.534565: step 7, loss 4.86689, acc 0.546875, precision 0.552632, recall 0.636364
temporal 2020-10-10T22:24:44.769879: step 8, loss 3.98059, acc 0.4375, precision 0.5, recall 0.361111
temporal 2020-10-10T22:24:45.026726: step 9, loss 2.85719, acc 0.609375, precision 0.733333, recall 0.564103
temporal 2020-10-10T22:24:45.260509: step 10, loss